<a href="https://colab.research.google.com/github/BugPersonality/ML/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

Без использования merge!

In [108]:
trTypes = pd.read_csv(filepath_or_buffer = ".vscode/csv/tr_types.csv", sep = ';')
trMccCode = pd.read_csv(filepath_or_buffer = ".vscode/csv/tr_mcc_codes.csv", sep = ';')
genderTrain = pd.read_csv(filepath_or_buffer = ".vscode/csv/gender_train.csv", sep = ',')
transactions = pd.read_csv(filepath_or_buffer = ".vscode/csv/transactions.csv", sep = ',').head(1000000)

In [111]:
def isContainsATM(string):
    return string.str.contains("АТМ").bool()

def isContainsPOS(string):
    return string.str.contains("POS").bool()

sample = transactions["tr_type"].sample(n = 1000)
count = 0

for i in sample: 
    description = trTypes.query(f'tr_type == {i}')['tr_description']
    if isContainsATM(description) or isContainsPOS(description):
        count += 1

print(count/1000)

0.624


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [113]:
allTypes = transactions.groupby('tr_type').size()
topTenTypes = pd.DataFrame(allTypes.sort_values(ascending=False).head(10).reset_index(name = 'freq'))

for index, row in topTenTypes.iterrows():
    trTypeInRow = row['tr_type']
    descriptionInTrTypeDF = trTypes.query(f'tr_type == {trTypeInRow}')['tr_description'].to_string(index = False)
    print(row['freq'], end=" ")
    print(row['tr_type'], end=" ")
    print(descriptionInTrTypeDF)

231117 1010 Покупка. POS ТУ СБ РФ
151166 2010 Выдача наличных в АТМ Сбербанк России
149006 7070 Перевод на карту (с карты) через Мобильный банк...
137658 1110 Покупка. POS ТУ Россия
118975 1030 Оплата услуги. Банкоматы СБ РФ
49830 2370 Списание с карты на карту по операции <перевода...
28972 7010 Взнос наличных через АТМ (в своем тер.банке)
26078 7030 Перевод на карту (с карты) через АТМ (в предела...
15781 7071 Перевод на карту (с карты) через Мобильный банк...
15476 1100 Покупка. ТУ  Россия


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [114]:
positiveAmount = transactions.query('amount > 0').groupby(['customer_id'])['amount']
negativeAmount = transactions.query('amount < 0').groupby(['customer_id'])['amount']

customerWithMaxAmount = positiveAmount.sum().sort_values(ascending=False).head(1).reset_index(name = 'amount')
customerWithMinAmount = negativeAmount.sum().sort_values(ascending=True).head(1).reset_index(name = 'amount')

print(f"id: {customerWithMaxAmount['customer_id'][0]}, amount: {customerWithMaxAmount['amount'][0]}")
print(f"id: {customerWithMinAmount['customer_id'][0]}, amount: {customerWithMinAmount['amount'][0]}")
print(f"diff: {abs(customerWithMaxAmount['amount'][0] - customerWithMinAmount['amount'][0])}")

id: 70780820, amount: 1248114886.81
id: 70780820, amount: -1249952204.79
diff: 2498067091.6


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [115]:
for i in topTenTypes['tr_type']:
    firstMean = transactions.query(f'tr_type == {i}')['amount'].mean()
    firstMedian1 = transactions.query(f'tr_type == {i}')['amount'].median()
    print(f"type: {i}, mean: {firstMean}, median: {firstMedian1}")

customerIndex = customerWithMinAmount['customer_id'][0]
secondMean = transactions.query(f'customer_id == {customerIndex}')['amount'].mean()
secondMedian = transactions.query(f'customer_id == {customerIndex}')['amount'].median()

print(f"customer_id: {customerIndex}, mean: {secondMean}, median: {secondMedian}")

type: 1010, mean: -19784.748639650046, median: -7411.52
type: 2010, mean: -136077.62932504667, median: -44918.32
type: 7070, mean: 65569.83170020001, median: 11319.42
type: 1110, mean: -32119.330370773943, median: -11207.57
type: 1030, mean: -5320.980221811305, median: -2245.92
type: 2370, mean: -205418.24903170782, median: -44918.32
type: 7010, mean: 276391.7895961618, median: 112295.79
type: 7030, mean: 86104.33290896541, median: 13951.52
type: 7071, mean: 66806.82662252075, median: 3593.47
type: 1100, mean: -44061.8272615663, median: -10188.259999999998
customer_id: 70780820, mean: -20.694945765423416, median: 8803.99


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [117]:
transactions = pd.merge(transactions, genderTrain, how='left')
transactions = pd.merge(transactions, trMccCode, how='inner')
transactions = pd.merge(transactions, trTypes, how='inner')

print(transactions.shape)

(999584, 9)


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [118]:
zeroSpentMean= transactions.query('gender == 0.0').query('amount < 0')['amount'].mean()
oneSpentMean = transactions.query('gender == 1.0').query('amount < 0')['amount'].mean()

print(f"Spent diff: {abs(zeroSpentMean - oneSpentMean)}")

zeroIncomeMean = transactions.query('gender == 0.0').query('amount > 0')['amount'].mean()
oneIncomeMean = transactions.query('gender == 1.0').query('amount > 0')['amount'].mean()

print(f"Income diff: {abs(zeroIncomeMean - oneIncomeMean)}")

Spent diff: 32718.054920224655
Income diff: 63366.57104801461


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [119]:
maxIncomesForZero = transactions.query('gender == 0.0').query('amount > 0').groupby('tr_type')['amount'].max().sort_values(ascending=False).reset_index(name = 'amount')
lessTenMaxIncomesForZero = maxIncomesForZero.tail(10)

maxIncomesForOne = transactions.query('gender == 1.0').query('amount > 0').groupby('tr_type')['amount'].max().sort_values(ascending=False).reset_index(name = 'amount')
lessTenMaxIncomesForOne = maxIncomesForOne.tail(10)

commonTrTypes = pd.merge(lessTenMaxIncomesForOne, lessTenMaxIncomesForZero, on = 'tr_type', how='inner')

print(f"type 0: \n{lessTenMaxIncomesForZero}")
print(f"type 1: \n{lessTenMaxIncomesForOne}")
print(f"common tr_types for 1 and 0: \n {commonTrTypes}")

type 0: 
    tr_type    amount
28     2210  68315.82
29     2010  44918.32
30     1010  24839.83
31     2110  22459.16
32     7075   6737.75
33     4100   5041.86
34     2370   2335.75
35     4210   2245.92
36     4110   2245.92
37     4051   1122.96
type 1: 
    tr_type     amount
24     7015  121279.45
25     2370  113194.15
26     2110  112295.79
27     2010   89836.63
28     7041   87590.72
29     6000   79280.83
30     8100   64682.37
31     1010   28803.87
32     4110    2245.92
33     2020    2245.92
common tr_types for 1 and 0: 
    tr_type   amount_x  amount_y
0     2370  113194.15   2335.75
1     2110  112295.79  22459.16
2     2010   89836.63  44918.32
3     1010   28803.87  24839.83
4     4110    2245.92   2245.92


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [120]:
zeroSpentByMccCode = transactions.query('gender == 0.0').query('amount < 0').groupby('mcc_code')['amount'].sum().sort_values(ascending=False)
oneSpentByMccCode = transactions.query('gender == 1.0').query('amount < 0').groupby('mcc_code')['amount'].sum().sort_values(ascending=False)

zeroSpentByMccCode = zeroSpentByMccCode.head(len(zeroSpentByMccCode)).reset_index(name = 'amount')
oneSpentByMccCode = oneSpentByMccCode.head(len(oneSpentByMccCode)).reset_index(name = 'amount1')

print(zeroSpentByMccCode.head())
print(oneSpentByMccCode.head())

merged = pd.merge(zeroSpentByMccCode, oneSpentByMccCode, on = 'mcc_code', how = 'inner')
merged['diff'] = (abs(merged['amount'] - merged['amount1']))
merged = merged.sort_values(by = ['diff'], ascending=False).head()

description = []
for code in merged.mcc_code.values:
    description.append(transactions.query(f'mcc_code == {code}')['mcc_description'].values[0])   
    
merged['mcc_description'] = description

print(merged)

   mcc_code    amount
0      7372  -4886.89
1      5193 -10151.54
2      7395 -10263.83
3      5072 -11386.79
4      5192 -12195.33
   mcc_code   amount1
0      9402  -4581.67
1      8398  -5688.88
2      7829 -13430.58
3      5967 -19315.33
4      7375 -30236.77
     mcc_code        amount       amount1          diff  \
167      6011 -4.958077e+09 -8.043939e+09  3.085862e+09   
166      4829 -3.513864e+09 -6.013017e+09  2.499153e+09   
164      6010 -2.612731e+08 -4.821311e+08  2.208580e+08   
148      5541 -5.162489e+07 -2.070574e+08  1.554326e+08   
102      5511 -5.095159e+06 -7.862393e+07  7.352877e+07   

                                       mcc_description  
167  Финансовые институты — снятие наличности автом...  
166                                  Денежные переводы  
164   Финансовые институты — снятие наличности вручную  
148                            Станции техобслуживания  
102  Легковой и грузовой транспорт — продажа, серви...  


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [121]:
transactions[['index', 'hour']] = transactions['tr_datetime'].str.split(" ", expand = True)
transactions['hour'] = transactions['hour'].str.slice(0, 2)

In [122]:
one = transactions.query('amount < 0').query('gender == 1.0').groupby('hour').size().reset_index(name = 'count').head(6)
zero = transactions.query('amount < 0').query('gender == 0.0').groupby('hour').size().reset_index(name = 'count').head(6)

print(f"gender: 1, count: {one['count'].sum()}")
print(one )
print(f"gender: 0, count: {zero['count'].sum()}")
print(zero)


gender: 1, count: 43017
  hour  count
0   00  35697
1   01    984
2   02    921
3   03   1191
4   04   1615
5   05   2609
gender: 0, count: 38827
  hour  count
0   00  29060
1   01    917
2   02   1289
3   03   1893
4   04   2252
5   05   3416
